#  Task 3
## Task 3(a), 3(b), 3(C) and 3(d)
In the following cell contains the solution for the 3(a), 3(b), 3(c) and 3(d)

3(a) - create a tuples of Specialization Space-(s) and Generalization Space-(g)

3(b) - more_general method  h1>=h2

3(c) - min_generalizations(h,x) 

3(d) - min_specializations(h,domains,x)

Note  -- In our program, we assumed like 

         '?' - Wild card
         
         '^' - Contradiction

One more thing, we have not used any "contradition value" for the attribute in the "min_specializations()", because these are incosistent with the s in the S, and remove while checking the condition. what we thought

In [ ]:
import copy

# method for creating maximum general hypothesis
def g_0(n):
    return ("?",)*n

# method for creating minimum specification hypothesis
def s_0(n):
    return ("^",)*n

# this method will return the 'true' if h1>=(g)h2 otherwise 'false'
def more_general(h1,h2):                                 # we are passing two hypothesis 
    for each_h1, each_h2 in zip(h1,h2):                  # each attribute value in h1 and h2
        if each_h1 != each_h2 and each_h1 != "?":        # checking condition like if h1 is not equal or has no attribute value '?'
            return False                                # returns 'false'
    
    return True                                          # otherwise 'true'


#this method returns hypothesis which is more general than the h(passing argument) and c(x) for each x/X is equal to h(x) = c(x) =1, consistency condition
def min_generalizations(h, x):           #  h,x are hypothesis and instance
    hyp = ()                             #  local variable for storing the hypothesis that are more general than h
    for each_h, each_x in zip(h, x):     #  for each item in h and x
        if each_h == each_x:             #  checking if each of h and x are equal, if true
            hyp += (each_h,)             #  add that attribute value to the new hypothesis
        elif each_h == "^":              #  else if  each of h is equal to "^", then
            hyp += (each_x,)             #  add the each item of h attribute to hyp(new hypothesis)
        else:                            #  else
            hyp += ("?",)                #  making that attribute value to wild card "?"
    return [hyp]                         # finally return the hypothesis


# this method returns the hypothesises which are more specific than the h(passing argument) and c(x)= h(x)=0, consistency condition
'''def min_specializations(h, domains, x):         # h,domains,x are hypothesis ,list with lists, 
                                                #each list in list is the set of possible values for that attribute and  instance
    hypo_list=[]                                # local list for storing
    for i in range(len(h)):                     # for i in range(len(h))  eg: i from 0 to 7(exclude)
        for each in domains[i]:                 # for each in domains[i] represents each value of the list of available values for that attribute
            if each != x[i]:                    # if each value is not equal to x[i], then only consistency condition holds
                s = copy.deepcopy(list(h))      # each time copying h to the s
                s[i] = each                     # changing the particular attribute value to opposite one
                hypo_list.append(tuple(s))      # adding this list to the local list(main list)
    return hypo_list   '''                         # returning the hypo_list

def min_specializations(h, domains, x):
    hypo_list = []
    for i in range(len(h)):
        if h[i] == '?':
            for each in domains[i]:
                if each != x[i]:
                    s = copy.deepcopy(list(h))
                    s[i] = each                     
                    hypo_list.append(tuple(s))
        else:
            s = copy.deepcopy(list(h))
            s[i] = '^'                     
            hypo_list.append(tuple(s))
    return hypo_list


#  this method is for checking consistency holds between given hypothesis and instance
#  for positive examples, we need the return should be True
#  for negative examples, we need the return should be False
#  remeber these two things, which drives our candidate elimination algorithm
def consistent(hypothesis,instance):             #  two arguments, hypothesis and instance
    for h, x in zip(hypothesis, instance):       #  for each item in hypothesis and instance
        if h != x and h != "?":                  #  either h is equal to x or h = "?"  (not holds)
            return False                         #  return False
    return True                                  #  return True



## Task 3 e:  Candidate Elimination Algorithm Implementation
Here, we simply follow the pseudo code in the lecture notes


In [ ]:
import copy

# method returns the G and S spaces from the passing example set
def candidate_elimination(examples):              # argument examples list of instances
    # step 1: initializing the maximum Generalization and maximum Specification hypothesis
    G = [g_0(len(examples[0])-1)]                 
    S = [s_0(len(examples[0])-1)]
    # defining the domains list, simply looping through each instance
    domains = []
    for i in range(len(examples[0])-1):
        l = []
        for each in examples:
            if not each[i] in l:
                l += [each[i],]
        domains.append(l)
    
    #  step 2: for each instance in the given list of instances
    for instance in examples:
        # if instance is positive,  c(x) = 1 
        if instance[len(instance)-1] == True:                                                                 
            # FOREACH g ∈ G DO IF g(x) not equal to 1 THEN G = G \ {g} ENDDO
            for g in G:                                                      
                if consistent(g,instance) != instance[len(instance)-1]:
                    G.remove(g)
                    
            for every in S:                                                   # FOREACH s ∈ S DO
                if consistent(every,instance) != instance[len(instance)-1]:   # IF s(x) 6= 1 THEN
                    S.remove(every)                                           # S = S \ {s}
                    S_plus = min_generalizations(every, instance)             # S+ = min_generalizations(s, x)                                         
                    for s in S_plus:                                          # # FOREACH s ∈ S+ DO IF (∃g ∈ G : g ≥g s) THEN S = S ∪ {s} ENDDO
                        yes = False                      
                        for g in G:                                         
                            if more_general(g,s):                         
                                yes = True
                                break
                        if yes:
                            S.append(s)
                    for s in S:             # FOREACH s ∈ S DO IF (∃s0 ∈ S : s0 not equal to s ∧ s ≥g s0) THEN S = S \ {s} ENDDO
                        yes = False
                        for each in S:
                            if each != s and more_general(s,each) :
                                yes = True
                                break
                        if yes:
                            S.remove(s)
        
        else:    # if instance is negative example c(x) = 0
            for s in S:                                                 # FOREACH s ∈ S DO IF s(x) not equal to 0 THEN S = S \ {s} ENDDO
                if consistent(s,instance) != instance[len(instance)-1]:
                    S.remove(s)
            for g in G:                                                 # FOREACH g ∈ G DO
                if consistent(g,instance) != instance[len(instance)-1]: # IF g(x) not equal to 0 THEN
                    G.remove(g)                                         # G = G \ {g}
                    G_minus = min_specializations(g,domains,instance)   # G− = min_specializations(g, x)
                    for g in G_minus:                                   # FOREACH g ∈ G− DO IF (∃s ∈ S : g ≥g s) THEN G = G ∪ {g} ENDDO
                        yes = False
                        for s in S:
                            if more_general(g,s):
                                yes = True
                                break
                        if yes:
                            G.append(g)
                    for g in G:                     # FOREACH g ∈ G DO IF (∃g0 ∈ G : g0 not equal to g ∧ g0 ≥g g) THEN G = G \ {g} ENDDO
                        yes = False
                        for each in G:
                            if each != g and more_general(each, g) :
                                yes = True
                                break
                        if yes:
                            G.remove(g)
    return tuple(G),tuple(S)                                           # step 3: return (G,S)

## Task - 3 f: Testing the Candidate Elimination algorithm with the provided example set

In [ ]:
examples = [("sunny","warm","normal","strong","warm","same","long",True),("sunny", "warm", "high", "strong", "warm", "same", "long", True),
            ("rainy", "cold", "high", "weak", "warm", "change", "medium", False),("sunny", "warm", "high", "strong", "cool", "change", "long", True),
           ("cloudy", "cold", "high", "strong", "cool", "change", "short", False)]
general, specific = candidate_elimination(examples)
print("\nG[5]: ", general,"\n\n")
print("S[5]: ", specific)

## Task 3 g:  Version space function
we created seperate method for version space to visualize it

In [14]:
import copy

# this method is for to setting the break condition for the while in the versionspace function, but unfortunately , it does work
# so we did not use this method, it creates endless loop
def breakcondition(version_list, s):
    if len(version_list) == 0:
        return True
    for each in version_list:
        if each != s:
            return True
    return False

# checking the whether given list of lists has the particular list in it
# this method is for eliminating the repeation in the version space 
def checking(space, item):    #  here, space is the list of lists and item is testiong list(whether it presents in the 'space')
    if len(space) == 0:
        return True
    for each in space:
        if item == each:
            return False
    return True
# this method is for checking two lists are equal or not
def twolists(list_a,list_b):              
    for a,b in zip(list_a,list_b):
        if a!=b:
            return False
    return True

# this method returns the versionspace for the provided G and S
# this can work only for the given set of examples problem, because my breakcondition creates endless loop,
# if that will be eliminated, it works for any G and S 
def versionspace(G,S):
    version_list = []
    final_version_space = []
    duplicate = []
    for s in S:
        duplicate = copy.deepcopy(list(G))
        #while breakcondition(version_list,s):
        i = 0
        while i < 2:
            version_list.clear()
            for g in duplicate:
                for each in enumerate(zip(g,s)):
                    lis = copy.deepcopy(list(g))
                    # here 'each' is the list,looks like [0,[g,s]]
                    # that,s why i represented below item values like that
                    if each[1][0] != each[1][1] and each[1][0] == "?":
                        lis[each[0]] = each[1][1]
                        if checking(version_list,lis) and checking(final_version_space,lis) and not twolists(lis,s):
                            version_list.append(lis)
            duplicate = copy.deepcopy(version_list)     # local storing list, every time, it will be cleared, at the starting of the loop
            final_version_space += version_list         # final version list
            i = i+1
    return final_version_space

## Task 3 g:  Visualization of Version Space

In [15]:

print("S: ", specific,"\n\n\n")
print("Version_Space:\n",versionspace(general,specific),"\n\n\n")
print("G: ", general,"\n\n\n")

S:  (('sunny', 'warm', '?', 'strong', '?', '?', 'long'),) 



Version_Space:
 [['sunny', 'warm', '?', '?', '?', '?', '?'], ['sunny', '?', '?', 'strong', '?', '?', '?'], ['sunny', '?', '?', '?', '?', '?', 'long'], ['?', 'warm', '?', 'strong', '?', '?', '?'], ['?', 'warm', '?', '?', '?', '?', 'long'], ['?', '?', '?', 'strong', '?', '?', 'long'], ['sunny', 'warm', '?', 'strong', '?', '?', '?'], ['?', 'warm', '?', 'strong', '?', '?', 'long'], ['sunny', 'warm', '?', '?', '?', '?', 'long'], ['sunny', '?', '?', 'strong', '?', '?', 'long']] 



G:  (('sunny', '?', '?', '?', '?', '?', '?'), ('?', 'warm', '?', '?', '?', '?', '?'), ('?', '?', '?', '?', '?', '?', 'long'), ('?', 'warm', '?', 'strong', '?', '?', '?')) 



